In [1]:
import json, os
from moviepy.editor import VideoFileClip
from IPython.display import Video, display


def draw_rectangle(box):
    def draw(frame):
        '''Draw a rectangle in the frame'''
        fr = frame.copy()
        bottom = int(box['y1'])
        top = int(bottom + box['height'])
        left = int(box['x1'])
        right = int(left + box['width'])
        fr[top-5:top+5, left:right] = 255 - fr[top-5:top+5, left:right]
        fr[bottom-5:bottom+5, left:right] = 255 - fr[bottom-5:bottom+5, left:right]
        fr[bottom:top, left-5:left+5] = 255 - fr[bottom:top, left-5:left+5]
        fr[bottom:top, right-5:right+5] = 255 - fr[bottom:top, right-5:right+5]
        return fr
    return draw

def draw_keypoints(keypoints, fps, size = 5, threshold = 0):
    def draw(get_frame, t):
        '''Draw keypoints in the frame'''
        fr = get_frame(t).copy()
        keypoints_t: list[float] = keypoints[min(int(t * fps), len(keypoints)-1)]['keypoints']
        it = iter(keypoints_t)
        for x, y, conf in list(zip(it, it, it))[94:]:
            if conf > threshold:
                fr[int(y)-size:int(y)+size:,int(x)-size:int(x)+size] = 255-fr[int(y)-size:int(y)+size:,int(x)-size:int(x)+size]
        return fr
    return draw

def group_kds(kds):
    '''Groups keypoint data objects that belong to same frame'''
    grouped = [[]]
    for kd in kds:
        added = False
        for g in grouped:
            if not added and (not g or g[-1]['image_id'] != kd['image_id']):
                g.append(kd)
                added = True
        if not added:
            grouped.append([kd])
    return grouped

def update_clip(clip, size, threshold, output = "temp"):
    rec_clip = VideoFileClip(clip+".mp4")
    with open(clip+"_signer.json") as signerf:
        signer = json.load(signerf)
    with open(clip+"_ap.json") as apf:
        ap = json.load(apf)
    signers = group_kds(ap)
    for s in signers:
        rec_clip = rec_clip.fl(draw_keypoints(s, rec_clip.fps, size, threshold))
    final_clip = rec_clip.fl_image(draw_rectangle(signer['roi']))
    final_clip.write_videofile(f"../data/temp/{output}.mp4", codec="libx264", fps=15, verbose=False, logger=None)
    with open(clip+".json") as dataf:
        data = json.load(dataf)
    return data

In [2]:
def get_score(scores: list[float]):
    m1 = max(scores)
    scores = scores.copy()
    scores.remove(m1)
    return (m1 - max(scores)) / m1

path = "../data/cuts/"
cuts = [(path+vid+'/'+cut[:-4]) for vid in os.listdir(path) for cut in os.listdir(path + vid) if cut.endswith(".mp4")]

by_signers = {"all": set()}
by_scores = {"all": set()}
for cut in cuts:
    by_scores["all"].add(cut)
    by_signers["all"].add(cut)
    with open(cut + "_signer.json") as datafile:
        line = ""
        while ']' not in line:
            line += datafile.readline()
        line = line[:-2] + '}'
    scores = json.loads(line)["scores"]
    by_signers.setdefault(str(len(scores)),set()).add(cut)
    if len(scores) > 1:
        if max(scores) == 0:
            # Error en video que solo se ven imagenes en carteles
            print(cut)
            continue
        by_scores.setdefault(str(round(get_score(scores), 2)),set()).add(cut)

print(by_signers.keys())
print(len(by_scores.keys()))

../data/cuts/lsa-noticias-en-lengua-de-senas-argentina-programa-especial-20062021/237
../data/cuts/noticias-en-lengua-de-senas-argentina-resumen-semanal-07032021/213
../data/cuts/lsa-noticias-en-lengua-de-senas-argentina-resumen-semanal-23052021/175
../data/cuts/leyfederalsa-resumen-de-la-mesa-de-trabajo-region-patagonia/111
../data/cuts/lsa-noticias-en-lengua-de-senas-argentina-resumen-semanal-30052021/119
../data/cuts/lsa-noticias-en-lengua-de-senas-argentina-resumen-semanal-07062021/205
../data/cuts/noticias-en-lengua-de-senas-argentina-resumen-semanal-03012021/307
../data/cuts/noticias-en-lengua-de-senas-argentina-resumen-semanal-03012021/239
../data/cuts/noticias-en-lengua-de-senas-argentina-resumen-semanal-31012021/527
../data/cuts/noticias-en-lengua-de-senas-argentina-resumen-semanal-semana-santa-04042021/433
../data/cuts/noticias-en-lengua-de-senas-argentina-resumen-semanal-semana-santa-04042021/448
../data/cuts/noticias-en-lengua-de-senas-argentina-resumen-semanal-semana-santa

In [4]:
SIGNERS = "all"
SCORES = "1.0"

AMOUNT = 5
OFFSET = 5

for i, c in enumerate(list(by_signers[SIGNERS] & by_scores[SCORES])[OFFSET:OFFSET + AMOUNT]):
    data = update_clip(c, 7, 0.5, i)
    print(f"Video: {c}")
    print(f"Label: {data['label']}")
    display(Video(f"../data/temp/{i}.mp4", height=300))

Video: ../data/cuts/noticias-en-lengua-de-senas-argentina-resumen-semanal-7022021/151
Label: les quería contar del día que venía manejando hacia mi casa,


Video: ../data/cuts/lsa-noticias-en-lengua-de-senas-argentina-resumen-semanal-23052021/168
Label: aportar positivamente.


Video: ../data/cuts/mascotas-en-la-ciudad-episodio-9-en-lengua-de-senas-argentina-lsa/1
Label: mi nombre es luciana.


Video: ../data/cuts/lsa-noticias-en-lengua-de-senas-argentina-resumen-semanal-23052021/177
Label: ahora, con el proyecto de la ley federal ¿qué objetivo a futuro tenés?


Video: ../data/cuts/noticias-en-lengua-de-senas-argentina-resumen-semanal-11042021/133
Label: con la rendición de la argentina y provocó la muerte de 649 soldados argentinos,


In [37]:
clip = path + input("Clip:")

data = update_clip(clip, 7, 0.5)

print(f"Video: {clip}")
print(f"Label: {data['label']}")
display(Video("../data/temp/temp.mp4", height=300))

Video: ../data/cuts/noticias-en-lengua-de-senas-argentina-resumen-semanal-28022021/85
Label: es muy importante, llevar una dieta saludable y la actividad física suficiente


In [ ]:
error = input(f"Crop error in video {clip}? y/[n]")
if error == 'y':
    with open("../data/errors.csv", 'a') as errors_f:
        errors_f.write(clip[:-1] + ',')
if os.path.isfile("../data/errors.csv"):
    with open("../data/errors.csv", 'r') as errors_f:
        for f in errors_f.read().split(','):
            print(f)